## Instalacion de librerias 

In [ ]:
%pip install pandas
%pip install sodapy
import pandas as pd
from sodapy import Socrata

## carga de datos

se cargan los datos desde la api, la cual en nuestra arquitectura de nube vendria a ser nuestro data lake.

In [26]:
def load_data():
    warnings.filterwarnings("ignore")

    client = Socrata("www.datos.gov.co", None)

    results = client.get("vw9t-pugy", limit=8000)

    datos = pd.DataFrame.from_records(results)

    return datos

## seleccion de columnas

**Los datos vienen inicialmente con las siguientes columnas**

- **codigo** = codigo identificador	
- **ideps** = identificador unico
- **eps**	= eps a la que esta inscrita la persona
- **nomcategorias** = categoria del indicador de calidad
- **nomservicio**	= tipo de servicio
- **nomespecifique** = resultado esperado del servicio
- **nomindicador** = indicador de calidad	
- **resultado** = tiempo en que se resolvio el servicio	
- **nomunidad** = unidad en que se mide la variable resultado, se mide en dias
- **nomfuente** = fuente de los datos	
- **fecha_corte**	= fecha de estudio

se seleccionan las columnas que son importantes para el analisis

In [27]:
def select_columns(datos):
    droped_cols = [
        "codigo", # no es util para el analisis
        "ideps", # no es util para el analisis
        "nomcategorias", #redundante ya que es el mismo para todos
        "nomunidad", #redundante ya que es el mismo para todos
        "nomfuente", # no es util para el analisis
        "linkfuente", # no es util para el analisis
    ]

    return datos.drop(columns=droped_cols)

## limpieza de los datos

se evalua que informacion requiere una limpieza

In [28]:
datos = load_data()
datos = select_columns(datos)

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7785 entries, 0 to 7784
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   eps             7785 non-null   object
 1   nomservicio     7785 non-null   object
 2   nomespecifique  7785 non-null   object
 3   nomindicador    7785 non-null   object
 4   resultado       7785 non-null   object
 5   fecha_corte     7785 non-null   object
dtypes: object(6)
memory usage: 365.1+ KB


#### se hicieron los siguientes hallazgos:

 - **FECHA**: no tiene el tipo de dato correcto, se convierte a tipo fecha
 - **resultado**: no tiene el tipo de dato correcto, se convierte a tipo numerico

In [29]:
def clean_data(datos):
    # eliminar datos duplicados
    datos.drop_duplicates(inplace=True)

    # convertir fechas a tipo de dato datetime
    datos["fecha_corte"] = pd.to_datetime(datos["fecha_corte"], infer_datetime_format=True)
    
    #convertir el campo resultado a numerico
    datos["resultado"] = pd.to_numeric(datos["resultado"], errors="coerce")   
    return datos

## datos limpios

In [30]:
def get_data():
    datos = load_data()
    datos = select_columns(datos)
    datos = clean_data(datos)

    return datos

In [ ]:
datos = get_data()
datos.info()